## **Contents**
-  [Introduction](#i)
-  [1.Importing Libraries](#1)
-  [2.Helper Functions](#2)
-  [3.Dataset Managament](#3)
    -  [3.1.Dataset and DataLoaders](#3.1)
    -  [3.2.Visualizing Dataset](#3.2)
-  [4.Initializing pre-trained model](#4)
-  [5.Training](#5)
-  [6.Plotting Graphs](#6)
    -  [6.1.Plotting Loss vs Epoch](#6.1)
-  [7.Loading and Testing](#7)
-  [8.Predictions](#8)

## **Introduction** <a class="anchor" id="i"></a>


In this notebook I am visualizing the dataset and also training [U-Net](https://arxiv.org/abs/1505.04597) model using [Segmentation model](https://github.com/qubvel/segmentation_models.pytorch) library which is based on [Pytorch](https://pytorch.org/) library.  
Resnet34 net is being utilized for the U-Net trained in this notebook.

U-Net is an architecture for semantic segmentation. It consists of a contracting path and an expansive path. The contracting path follows the typical architecture of a convolutional network. It consists of the repeated application of two 3x3 convolutions (unpadded convolutions), each followed by a rectified linear unit (ReLU) and a 2x2 max pooling operation with stride 2 for downsampling. At each downsampling step we double the number of feature channels. Every step in the expansive path consists of an upsampling of the feature map followed by a 2x2 convolution (“up-convolution”) that halves the number of feature channels, a concatenation with the correspondingly cropped feature map from the contracting path, and two 3x3 convolutions, each followed by a ReLU. The cropping is necessary due to the loss of border pixels in every convolution. At the final layer a 1x1 convolution is used to map each 64-component feature vector to the desired number of classes. In total the network has 23 convolutional layers.


Read more about U-Net [here](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

Some of the helper functions is derived from [this](https://www.kaggle.com/julian3833/sartorius-starter-baseline-torch-u-net/notebook#Model:-U-net) notebook.

<img src= "https://miro.medium.com/max/953/1*Z98NhzbVISHa4CoemZS4Kw.png"  style='width: 700px;'>

### **1.Importing Libraries** <a class="anchor" id="1"></a>

In [ ]:
%%capture
!pip install ../input/segmentation-models-wheels/efficientnet_pytorch-0.6.3-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/pretrainedmodels-0.7.4-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/timm-0.3.2-py3-none-any.whl
!pip install ../input/segmentation-models-wheels/segmentation_models_pytorch-0.1.3-py3-none-any.whl

In [ ]:
import os
import torch
import random
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
import seaborn as sns
from PIL import Image
from PIL import Image
from sklearn import cluster
import torch.nn.functional as F
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset as Dataset
from torch.utils.data import DataLoader as DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from albumentations import (HorizontalFlip, VerticalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)

torch.cuda.empty_cache()
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

sns.set_style("darkgrid")

### **2.Helper Functions** <a class="anchor" id="2"></a>

In [ ]:
IMG_SIZE = (224,224)
root = "../input/sartorius-cell-instance-segmentation/"

def rle_decode(mask_rle, shape, color=1):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

def build_masks(df_train, image_id, input_shape):
    height, width = input_shape
    labels = df_train[df_train["id"] == image_id]["annotation"].tolist()
    mask = np.zeros((height, width))
    for label in labels:
        mask += rle_decode(label, shape=(height, width))
    mask = mask.clip(0, 1)
    return mask

def post_process(probability, threshold=0.5, min_size=300):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = []
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            a_prediction = np.zeros((520, 704), np.float32)
            a_prediction[p] = 1
            predictions.append(a_prediction)
    return predictions

# Stolen from: https://www.kaggle.com/arunamenon/cell-instance-segmentation-unet-eda
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
# Modified by me

def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

def check_is_run_length(mask_rle):
    if not mask_rle:
        return True
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    start_prev = starts[0]
    ok = True
    for start in starts[1:]:
        ok = ok and start > start_prev
        start_prev = start
        if not ok:
            return False
    return True

def create_empty_submission():
    fs = os.listdir("../input/sartorius-cell-instance-segmentation/test")
    df = pd.DataFrame([(f[:-4], "") for f in fs], columns=['id', 'predicted'])
    df.to_csv("submission.csv", index=False)

### **3.Dataset Managament** <a class="anchor" id="3"></a>

#### 1.Downloading and Extracting Dataset <a class="anchor" id="3.1"></a>

In [ ]:
class SatoriusDataset(torch.utils.data.Dataset):
    def __init__(self, transforms, root = 'data/',train = True):
        self.root = root
        self.transforms = transforms
        self.w , self.h = 520 , 704
        info = pd.read_csv(self.root+'train.csv')[["id","annotation"]]
        info = info.groupby('id')['annotation'].agg(lambda x: list(x)).reset_index()
        validation =  30
        self.data_info = 0

        if train == True:
            self.data_info = info[validation:].reset_index(drop=True)
        else:
            self.data_info = info[:validation].reset_index(drop=True)
            
    def __getitem__(self, idx):
        img = Image.open(self.root+'train/'+self.data_info['id'][idx]+'.png').convert("RGB")
        labels = self.data_info['annotation'][idx]
        mask = np.zeros((self.w , self.h))
        for label in labels:
            mask = mask + rle_decode(label, shape=(self.w , self.h))
        mask = mask.clip(0, 1)
        mask = (mask >= 1).astype('float32')
        augmented = self.transforms(image=np.array(img), mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        return img, mask.reshape((1, IMG_SIZE[0], IMG_SIZE[1]))

    def __len__(self):
        return len(self.data_info)


    
def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(Resize(IMG_SIZE[0], IMG_SIZE[1]))
    transforms.append(Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)))
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(HorizontalFlip(0.5))
        transforms.append(VerticalFlip(0.5))
        
    transforms.append(ToTensorV2())
    return Compose(transforms)


train_dataset = SatoriusDataset(transforms=get_transform(train=True),root =root,train = True)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

validation_dataset = SatoriusDataset(transforms=get_transform(train=False),root =root,train = False)
validation_dataloader = DataLoader(validation_dataset, batch_size=30, shuffle=False)

#### 2.Visualizing Dataset <a class="anchor" id="3.2"></a>

In [ ]:
train_df = train_dataset.data_info
validation_df = validation_dataset.data_info

pd.set_option('max_colwidth', 125)
display(train_df.head())
display(validation_df)

In [ ]:
sns.set_palette("pastel")

n_annotations = [[],[]] #[[train], [validation]]
for i in train_df['annotation']:
    n_annotations[0].append(len(i))
    
for i in validation_df['annotation']:
    n_annotations[1].append(len(i))

x_axis,y_axis = 'Number Of Annotations' , 'Number Of Images'

fig = plt.figure(figsize=(8,8))
p = sns.histplot(data = n_annotations[0])
p.set_xlabel(x_axis, fontsize = 15)
p.set_ylabel(y_axis, fontsize = 15)


plt.show()

fig = plt.figure(figsize=(8,8))
p = sns.histplot(data = n_annotations[1])
p.set_xlabel(x_axis, fontsize = 15)
p.set_ylabel(x_axis, fontsize = 15)
plt.show()

In [ ]:
import cv2

img, mask = next(iter(validation_dataloader))
print(f'Shape of Image : {img.shape} and Shape of Mask : {mask.shape}')
image = np.array(img[0])[0]
mask = mask[0].reshape((IMG_SIZE[0], IMG_SIZE[1]))

plt.figure(figsize=(10, 10))
plt.imshow(image,cmap = 'plasma')
plt.axis("off")
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.imshow(mask,alpha = 0.3,cmap = 'plasma')
plt.axis("off")
    
plt.show()

In [ ]:
def clustered_img(x):
    kmeans = cluster.KMeans(2)
    dims = np.shape(x)
    pixel_matrix = np.reshape(x, (dims[0] * dims[1], dims[2]))
    clustered = kmeans.fit_predict(pixel_matrix)
    clustered_img = np.reshape(clustered, (dims[0], dims[1]))
    return clustered_img

for i in range(1):
    fig = plt.figure(figsize=(10,10))
    img, mask = next(iter(validation_dataloader))
    img = img[0]
    mask = mask[0].reshape((IMG_SIZE[0], IMG_SIZE[1]))
    plt.imshow(clustered_img(img.numpy().transpose((1,2,0))))
    plt.grid(None)
    plt.title("K-Means")
    plt.show()

    fig = plt.figure(figsize=(10,10))
    plt.imshow(img.numpy()[0],cmap = 'plasma')
    plt.imshow(mask, alpha=0.3)
    plt.grid(None)
    plt.title("Target")
    plt.show()

### **4.Initializing pre-trained model** <a class="anchor" id="4"></a>

In [ ]:
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/pytorch-pretrained-image-models/resnet34.pth /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth

import torch
import collections.abc as container_abcs
torch._six.container_abcs = container_abcs
import segmentation_models_pytorch as smp

In [ ]:
import segmentation_models_pytorch as smp
model = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)

In [ ]:
device = 'cuda'
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

epoch = 27
loss_history = [[],[]]
train_n_minibatches = train_dataloader.__len__()
validation_n_minibatches = validation_dataloader.__len__()

In [ ]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))


class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()


class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()
    
criterion = MixedLoss(10.0, 2.0)

### **5.Training** <a class="anchor" id="5"></a>

In [ ]:
model.train()
log_idx = 3

for e in range(epoch):
    for batch_idx , (images ,masks) in enumerate(train_dataloader):
        optimizer.zero_grad()
        images, masks = images.to(device),  masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        loss_history[0].append(float(loss.detach()))
        
        if batch_idx % log_idx == 0:
            # Printing Log
            print(f'LOSS for EPOCH {e+1} BATCH {batch_idx+1}/{train_n_minibatches} TRAIN LOSS : {loss_history[0][-1]}',end = ' ')
            with torch.no_grad():
                # Calculating loss and accuracy for validation
                for _batch_idx_ , (images ,masks) in enumerate(validation_dataloader):
                    images, masks = images.to(device),  masks.to(device)
                    outputs = model(images)
                    validation_loss = criterion(outputs, masks)
                    loss_history[1].append(float(validation_loss.detach()))
                                      
                print(f'VALIDATION LOSS : {sum(loss_history[1][-1:-validation_n_minibatches-1:-1])/validation_n_minibatches}')

    torch.save(model.state_dict(),'unet_ss')
    #Log for e+1th epoch
    print(f'---------------------------------------EPOCH {e+1}-------------------------------------------')
    print(f'Loss for EPOCH {e+1}  TRAIN LOSS : {sum(loss_history[0][-1:-train_n_minibatches-1:-1])/train_n_minibatches}')
    n_validation_losses = int(train_n_minibatches/log_idx)*validation_n_minibatches
    print(f'VALIDATION LOSS for EPOCH {e+1} : {sum(loss_history[1][-1:-1*n_validation_losses-1:-1])/n_validation_losses}',end = '\n')
    print('---------------------------------------------------------------------------------------------')

### **6.Plotting Graphs** <a class="anchor" id="6"></a>

#### 1.Plotting Loss vs Epoch<a class="anchor" id="6.1"></a>

In [ ]:
# Plotting Loss per epoch
loss_per_epoch = [[],[]]
for i in range(epoch):
    temp = 0
    for j in loss_history[0][i*train_n_minibatches:(i+1)*train_n_minibatches]:
        temp = temp + j
    loss_per_epoch[0].append(temp/train_n_minibatches)
    temp = 0
    for j in loss_history[1][i*n_validation_losses:(i+1)*n_validation_losses]:
        temp = temp + j
    loss_per_epoch[1].append(temp/n_validation_losses)    

sns.lineplot(x=range(len(loss_per_epoch[0])),y=loss_per_epoch[0])
sns.lineplot(x=range(len(loss_per_epoch[1])),y=loss_per_epoch[1])
plt.show()

### **7.Loading and Testing**<a class="anchor" id="7"></a>

In [ ]:
# model = smp.Unet("resnet34", encoder_weights="imagenet", activation=None)
# model.to(device)
# model.load_state_dict(torch.load('unet_ss', map_location='cuda'))
with torch.no_grad():
    for _batch_idx_ , (images ,masks) in enumerate(validation_dataloader):
        images, masks_target = images.to(device),  masks.to(device)
        masks_out = model(images)

images = images.cpu().detach().numpy()
masks_out = masks_out.cpu().detach().numpy()
masks_target = masks_target.cpu().detach().numpy()

for i in range(1):
    image = images[i][0]
    mask_out = masks_out[i].reshape((IMG_SIZE[0], IMG_SIZE[1]))
    mask_target = masks_target[i].reshape((IMG_SIZE[0], IMG_SIZE[1]))
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image , cmap = 'bone')
    plt.title("Image")
    plt.axis("off")
    plt.show()
    
    plt.figure(figsize=(8, 8))
    plt.imshow(mask_target , cmap = 'bone')
    plt.title("Ground Truth")
    plt.axis("off")
    plt.show()
    
    plt.figure(figsize=(8, 8))
    plt.imshow(mask_out , cmap = 'bone')
    plt.title("Prediction")
    plt.axis("off")
    plt.show()

### **8.Predictions**<a class="anchor" id="8"></a>

In [ ]:
class TestDataset(Dataset):
    def __init__(self, root,transforms = None):
        self.root = root
        self.transforms = transforms
        self.img_name = []
        for i in os.listdir(root+'test/'):
            self.img_name.append(i[:-4])
        self.w , self.h = 520 , 704

            
    def __getitem__(self, idx):
        image = Image.open(self.root+'test/'+self.img_name[idx]+'.png').convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image=np.array(image))['image']
        
        return {'image': image, 'id': self.img_name[idx]}

    def __len__(self):
        return len(self.img_name)
    

td = TestDataset(root, transforms=get_transform(train=False))
test_loader = DataLoader(td, batch_size=3, shuffle=False)

In [ ]:
model.eval()

submission = []
for i, batch in enumerate(test_loader):
    preds = torch.sigmoid(model(batch['image'].cuda()))
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    for image_id, probability_mask in zip(batch['id'], preds):
        try:
            #if probability_mask.shape != IMAGE_RESIZE:
            #    probability_mask = cv2.resize(probability_mask, dsize=IMAGE_RESIZE, interpolation=cv2.INTER_LINEAR)
            probability_mask = cv2.resize(probability_mask, dsize=(704, 520), interpolation=cv2.INTER_LINEAR)
            predictions = post_process(probability_mask)
            for prediction in predictions:
                #plt.imshow(prediction)
                #plt.show()
                try:
                    submission.append((image_id, rle_encoding(prediction)))
                except:
                    print("Error in RL encoding")
        except Exception as e:
            print(f"Exception for img: {image_id}: {e}")
        
        # Fill images with no predictions
        image_ids = [image_id for image_id, preds in submission]
        if image_id not in image_ids:
            submission.append((image_id, ""))
            
df_submission = pd.DataFrame(submission, columns=['id', 'predicted'])
df_submission.to_csv('submission.csv', index=False)

if df_submission['predicted'].apply(check_is_run_length).mean() != 1:
    print("Check run lenght failed")
    create_empty_submission()



In [ ]:
df_submission.groupby('id')['predicted'].agg(lambda x: list(x)).reset_index()